In [0]:
#1
# Install ADAL (Azure Active Directory Authentication Library)
%pip install adal

In [0]:
#Import the tools
#from env import env, not used for this notebook
from src import utils, excel

import openpyxl
import pandas as pd

from pyspark.sql import functions as F
from datetime import datetime
from openpyxl.styles import NamedStyle

In [0]:
# 2 A pop up will require the key code for authentication

# Mart authentication
# Note: 'adal' is deprecated; consider using 'msal' for long-term use.

import adal  # ← Required import

# Set MS context
tenant = "common" 
authority = f"https://login.microsoftonline.com/{tenant}" 
context = adal.AuthenticationContext(authority)

# Client ID for Azure Cloud Shell
client_id = "04b07795-8ddb-461a-bbee-02f9e1bf7b46" 
resource_app_id_url = "https://database.windows.net/" 

# Acquire user code
code = context.acquire_user_code(resource_app_id_url, client_id)
print(code['message'])  # Instructs user to authenticate

# Acquire token with device code
token = context.acquire_token_with_device_code(resource_app_id_url, code, client_id)
access_token = token["accessToken"]



In [0]:
# 3

# Read from mart, note table name is "Schema.TableName"

# JDBC driver and connection info
driver = "com.microsoft.sqlserver.jdbc.spark"
url = "jdbc:sqlserver://udalsqlmartprod.database.windows.net;databaseName=udal-sql-mart-qpat"
table_name = "EROC.PEROC_Initial_Validation"

# Read table into Spark DataFrame
df_raw  = spark.read \
       .format("jdbc") \
       .option("url", url) \
       .option("dbtable", table_name) \
       .option("accessToken", access_token) \
       .option("encrypt", "true") \
       .option("hostNameInCertificate", "*.database.windows.net") \
       .load()

In [0]:
#df_raw.display()

In [0]:
df_moved_discharged = (
    df_raw
    .where(F.col("EROC_DerMetric").isin(
        "PIFUSRTMV01", "PIFUSRTDC01", "PIFUTOTAL01", "PIFUCOMP001", "PIFUBOOK001", "PIFUDNA0001"
    ))
    .where(F.col("EROC_DerMonth") > '2023-03-01')
    #.where(F.col("EROC_Latest_Flag") == 1)
    .select(
        "EROC_DerMonth",
        "EROC_DerProviderCode",
        "EROC_DerTFC",
        "RegionName",
        "EROC_DerMetric",
        "EROC_Value",
        "EROC_Latest_Flag",
        "Submission_type"  
    )
    .orderBy("EROC_DerMonth", "EROC_DerProviderCode")
)

display(df_moved_discharged)

In [0]:
df_moved_discharged.write.format('delta').mode('overwrite').option('mergeSchema', 'true').option('overwriteSchema', 'true').save('abfss://analytics-projects@udalstdataanalysisprod.dfs.core.windows.net/ElectiveRecovery/Projects/PIFU_Validation')

In [0]:
#df_moved_discharged.display()
